In [72]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, f1_score, mean_absolute_error, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler,MinMaxScaler
import joblib


In [73]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [74]:
train.head(10)

,ID,Candidate,Constituency ∇,Party,Criminal Case,Total Assets,Liabilities,state,Education
0,0,M.K. Mohan,ANNA NAGAR,DMK,4,211 Crore+,2 Crore+,TAMIL NADU,8th Pass
1,1,Khatik Ramesh Prasad,KARERA (SC),BJP,0,1 Crore+,0,MADHYA PRADESH,12th Pass
2,2,Dr. Mantar Gowda,MADIKERI,INC,0,7 Crore+,22 Lac+,KARNATAKA,Post Graduate
3,3,Kundan Kumar,BEGUSARAI,BJP,0,9 Crore+,24 Lac+,BIHAR,Post Graduate
4,4,Swapan Majumder,BANGAON DAKSHIN (SC),BJP,2,2 Crore+,61 Lac+,WEST BENGAL,8th Pass
5,5,Arup Roy,HOWRAH MADHYA,AITC,0,3 Crore+,29 Lac+,WEST BENGAL,Graduate Professional
6,6,Baby Rani Maurya,AGRA RURAL (SC),BJP,0,2 Crore+,0,UTTAR PRADESH,Post Graduate
7,7,Neena Mittal,RAJPURA,AAP,1,9 Crore+,2 Crore+,PUNJAB,Graduate
8,8,Fareed Mahfooz Kidwai,RAM NAGAR,SP,1,5 Crore+,35 Lac+,UTTAR PRADESH,Post Graduate
9,9,Atul Moreshwar Save,AURANGBAD (EAST),BJP,6,22 Crore+,2 Crore+,MAHARASHTRA,Graduate


In [75]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
label_encoder = LabelEncoder()

In [76]:
education_mapping = {'Literate': 0, '5th Pass': 1, '8th Pass': 2, '10th Pass': 3, 
                     '12th Pass': 4, 'Graduate': 5, 'Post Graduate': 6, 
                     'Graduate Professional': 7, 'Doctorate': 8, 'Others': 9}

In [77]:
def getF1Score(model,X_val,y_val):
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred,average='weighted')
    print("F1 Score:", f1)  

In [78]:
def createSubmission(model,X_test,label_encoder):
    y_test=model.predict(X_test)
    df = pd.read_csv('test.csv')
    df = df[['ID']]
    # df['Education']=encoderEducation.classes_[y_test]
    # df['Education'] = [list(education_mapping.keys())[9] if i > 8 else list(education_mapping.keys())[i] for i in y_test]
    df['Education'] = label_encoder.inverse_transform(y_test)

    # Save the DataFrame to a new CSV file
    df.to_csv('output.csv', index=False)
    return y_test

In [79]:
def convert_financials(value):

    if isinstance(value, (float, int)):
        return value  
    value = value.replace('+', '').strip()  
    if 'Crore' in value:
        return float(value.replace('Crore', '')) * 10**7
    elif 'Lac' in value:
        return float(value.replace('Lac', '')) * 10**5
    elif 'Thou' in value:
        return float(value.replace('Thou', '')) * 10**3
    elif 'Hund' in value:
        return float(value.replace('Hund', '')) * 10**2
    return float(value)

In [80]:
def preprocess_features(data, encoder, fit_encoder = True):
    scaler=MinMaxScaler()
    data['Total Assets'] =  data['Total Assets'].apply(convert_financials)
    data['Liabilities'] =  data['Liabilities'].apply(convert_financials)

    data["Has Criminal Record"] = (data["Criminal Case"] > 0).astype(int) # binary variable
    data["Assets_liability"] =  (data['Total Assets']>data['Liabilities']).astype(int)# asset to liability ratio prop to education
    # data[["Assets_liability"]]=scaler.fit_transform(data[["Assets_liability"]])
    
    # data["Assets_liability"] = np.nan_to_num(data["Assets_liability"], nan=0)

    
    # financial distributions are skewed hence take log
    data['Log of Assets'] = np.log1p(data['Total Assets'])
    data['Log of Liabilities'] = np.log1p(data['Liabilities'])
    # data[['Log of Assets']]=scaler.fit_transform(data[['Log of Assets']])
    
    if encoder and fit_encoder:
        encoded_features = encoder.fit_transform(data[['Party', 'state']])
        encoded_data = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Party', 'state']))
        data = pd.concat([data.drop(['Party', 'state'], axis=1), encoded_data], axis=1)
    elif encoder:
        encoded_features = encoder.transform(data[['Party', 'state']])
        encoded_data = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Party', 'state']))
        data = pd.concat([data.drop(['Party', 'state'], axis=1), encoded_data], axis=1)

    return data
    
def preprocess_target(data, label_encoder=None, target_column=None, fit_encoder=False):
    if label_encoder and target_column:
        if fit_encoder:
            data[target_column] = label_encoder.fit_transform(data[target_column])
        else:
            pass
    return data

In [81]:
train_data = preprocess_features(train, encoder, fit_encoder=True)
train_data = preprocess_target(train_data, label_encoder, target_column='Education', fit_encoder=True)

test_data = preprocess_features(test, encoder, fit_encoder=False)
test_data = preprocess_target(test_data, label_encoder, target_column='Education', fit_encoder=False)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [82]:
train_data
train_data_0= train_data[train_data['Total Assets'] != 0]


In [98]:
X=train_data_0.drop(columns = ["ID", "Candidate", "Constituency ∇",'Total Assets','Liabilities', "Education","Criminal Case","Log of Liabilities","Log of Assets"], axis = 1)
y = train_data_0["Education"]

In [99]:
X

,Has Criminal Record,Assets_liability,Party_AAP,Party_AIADMK,Party_AITC,Party_BJD,Party_BJP,Party_CPI,Party_CPI(M),Party_DMK,...,state_ODISHA,state_PUDUCHERRY,state_PUNJAB,state_RAJASTHAN,state_SIKKIM,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL
0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2055,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2056,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2057,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
# import pandas as pd
# from scipy.stats import chi2_contingency

# # Assume 'data' is your DataFrame with 'Party' and 'Education level' columns

# # Create a contingency table (cross-tabulation)
# contingency_table = pd.crosstab(X['Log of Assets'], y)


# # Perform the chi-squared test
# chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# # Print the results
# print("Chi-squared statistic:", chi2_stat)
# print("p-value:", p_val)

KeyError: 'Log of Assets'

In [105]:
X_test=test_data.drop(columns = ["ID", "Candidate", "Constituency ∇",'Total Assets','Liabilities',"Criminal Case","Log of Liabilities","Log of Assets"], axis = 1)

In [102]:
X

,Has Criminal Record,Assets_liability,Party_AAP,Party_AIADMK,Party_AITC,Party_BJD,Party_BJP,Party_CPI,Party_CPI(M),Party_DMK,...,state_ODISHA,state_PUDUCHERRY,state_PUNJAB,state_RAJASTHAN,state_SIKKIM,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL
0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2055,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2056,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2057,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
# for i in range(500):    
rf = RandomForestClassifier(random_state=42)

# param_grid = {
#     'n_estimators': [100, 200, 300],  
#     'max_depth': [10, 20, 30, None],  
#     'min_samples_split': [2, 5, 10],  
#     'min_samples_leaf': [1, 2, 4]     
# }
param_grid = {
    'n_estimators': [100,150, 200, 300,400,500],  
    # 'max_depth': [10, 20, 30, None],  
    'min_samples_split': [2, 5, 7,9,11,13,15],  
    # 'min_samples_leaf': [1, 2, 4]  
    'max_leaf_nodes':[200,500,700]   
}

f1 = make_scorer(f1_score, average='weighted')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring=f1, n_jobs=-1)
# X = train_data.drop(columns = ["ID", "Candidate", "Constituency ∇", "Education"], axis = 1)
# y = train_data['Education']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation F1 score: {:.3f}".format(grid_search.best_score_))

y_pred = grid_search.predict(X_val)

print("Validation F1 Score: {:.3f}".format(f1_score(y_val, y_pred, average='weighted')))



Best parameters: {'max_leaf_nodes': 200, 'min_samples_split': 13, 'n_estimators': 300}
Best cross-validation F1 score: 0.215
Validation F1 Score: 0.232


In [106]:
best_rf=RandomForestClassifier(random_state=42,**grid_search.best_params_)
best_rf.fit(X,y)
createSubmission(best_rf,X_test,label_encoder)

array([1, 1, 9, ..., 5, 9, 9])

In [ ]:
# joblib.dump(best_rf, 'model_rf_24_200_5_300.pkl')